In [13]:
print("Importa librerie", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import keras_tuner as kt
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
import pickle
print(".", end="", flush=True)
from sklearn.model_selection import train_test_split
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)
print("ok")

Importa librerie........=
ok


In [14]:
print("Versione TensorFlow:", tf.__version__)
print("Versione CUDA:", tf.sysconfig.get_build_info()['cuda_version'])
print("Versione cuDNN:", tf.sysconfig.get_build_info()['cudnn_version'])
import psutil

def print_memory_usage():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

Versione TensorFlow: 2.14.0
Versione CUDA: 11.8
Versione cuDNN: 8


In [15]:
n_simboli_addestramento = "Tutti"
epochs=50
batch_size=100
n_timesteps = 120 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 1 # n. barre nel futuro di cui si desidera prevedere il prezzo
perc_dati = "dati"
set_file_x_y = f"_{n_simboli_addestramento}"
initial_lr = 0.001

features_prezzo = [
    "Close",
#    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "PSAR",
    "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
    "ATR",
    "PSARaf",
    "ADX",
]

features_meno_piu = [
    "MACDh",    
    "AROONOSC",
    "TRIX",
    "TRIXs",
    "DM_OSC",
]

features_no_scala = [
    "SUPERTd",  
    "PSARr"
]

features_candele = [
#    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
#    "EMA_5",
#    "EMA_20", 
#    "EMA_50",
    #"Open",
    #"High",
    #"Low",
    "Target"
]

col_features_prezzo = {col: idx for idx, col in enumerate(features_prezzo)}
col_features_da_scalare_singolarmente = {col: idx for idx, col in enumerate(features_da_scalare_singolarmente)}
col_features_meno_piu = {col: idx for idx, col in enumerate(features_meno_piu)}
col_features_no_scala = {col: idx for idx, col in enumerate(features_no_scala)}
col_features_candele = {col: idx for idx, col in enumerate(features_candele)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features_prezzo) + len(col_features_da_scalare_singolarmente) + len(col_features_meno_piu) + len(col_features_no_scala) + len(col_features_candele) 
n_targets = len(col_targets) 

def set_di_tuning(lista_ticker):
    if os.path.exists(f"{perc_dati}/indice.npy"):    
        inizio = np.load(f"{perc_dati}/indice.npy")
        print(inizio)
    else:
        inizio = 0

    if os.path.exists(f"{perc_dati}/X{set_file_x_y}.npy") and os.path.exists(f"{perc_dati}/Y{set_file_x_y}.npy"):
        print("Caricamento X e Y")
        X = np.load(f'{perc_dati}/X{set_file_x_y}.npy')
        Y = np.load(f'{perc_dati}/Y{set_file_x_y}.npy')
    else:
        X = np.zeros((0, n_timesteps, n_features))
        #Y = np.zeros((0, giorni_previsione, n_targets)) #togliere in caso di classificazione binaria
        Y = np.zeros((0, 1)) #solo per classificazione binaria
    if inizio < n_simboli_addestramento:
        lista_x, lista_y = [], []
        for i_ticker in range (inizio, n_simboli_addestramento):
            nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
            print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
            print("Download dati ticker", flush=True)
            try:
                ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
                if ticker["Close"].iloc[-1] >= 1:
                    ticker.index = ticker.index.date
                    print("Calcolo indicatori ticker", flush=True)
                    ticker = fx.crea_indicatori(ticker)
                    ticker.dropna(axis=0, inplace=True)

                    print("Definizione features e target", flush=True)
                    _, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                    print("Aggiunta dati a liste X e Y", flush=True)
                    lista_x.append(X_train)  
                    print(X_train.shape)      
                    lista_y.append(Y_train)   
                    print(Y_train.shape)
                    if ((i_ticker + 1) % 100 == 0):
                        print("Concatenamento su X, Y", flush=True)
                        X_arr = np.concatenate(lista_x, axis=0)
                        Y_arr = np.concatenate(lista_y, axis=0)
                        X = np.concatenate((X, X_arr), axis=0)
                        Y = np.concatenate((Y, Y_arr), axis=0)
                        print("Salvataggio X e Y su file", flush=True)
                        np.save(f'{perc_dati}/X{set_file_x_y}', X)
                        np.save(f'{perc_dati}/Y{set_file_x_y}', Y)
                        np.save(f"{perc_dati}/indice", i_ticker)
                        lista_x, lista_y = [], []
            except Exception as e:
                print(e)
                continue
                
        print("Concatenamento su X, Y", flush=True)
        X_arr = np.vstack(lista_x)
        Y_arr = np.vstack(lista_y)
        X = np.vstack((X, X_arr))
        Y = np.vstack((Y, Y_arr))
        print("Salvataggio X e Y su file", flush=True)
        np.save(f'{perc_dati}/X{set_file_x_y}', X)
        np.save(f'{perc_dati}/Y{set_file_x_y}', Y)
        np.save(f"{perc_dati}/indice", i_ticker)
        lista_x, lista_y = [], []

    return X, Y
        
def addestramento(lista_ticker):
    model = tf.keras.models.load_model("tuning.keras")
    
    if os.path.exists(f"{perc_dati}/indice_addestramento.npy"):    
        inizio = np.load(f"{perc_dati}/indice_addestramento.npy")
    else:
        inizio = 0
    print("ok")
    X = np.zeros((0, n_timesteps, n_features))
    #Y = np.zeros((0, giorni_previsione, n_targets)) #togliere in caso di classificazione binaria
    Y = np.zeros((0, 1)) #solo per classificazione binaria
    if inizio < n_simboli_addestramento:
        for i_ticker in range (inizio, n_simboli_addestramento):
            nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
            print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
            print("Download dati ticker", flush=True)
            try:
                ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
                if ticker["Close"].iloc[-1] >= 1:
                    ticker.index = ticker.index.date
                    print("Calcolo indicatori ticker", flush=True)
                    ticker = fx.crea_indicatori(ticker)
                    ticker.dropna(axis=0, inplace=True)

                    print("Definizione features e target", flush=True)
                    _, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                    print(f"Addestramento simbolo {nome_simbolo}", flush=True)
                    #model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val))
                    np.save(f"{perc_dati}/indice_addestramento", i_ticker)
                    model.save('addestramento.keras')

            except Exception as e:
                print(e)
                continue

def weighted_binary_crossentropy(pos_weight):
    def loss(y_true, y_pred):
        logloss = tf.keras.backend.mean(tf.keras.backend.binary_crossentropy(y_true, y_pred), axis=-1)
        return logloss * (1 + (pos_weight - 1) * y_true)
    return loss

class MixedHyperModel(kt.engine.hypermodel.HyperModel):
    def __init__(self, n_timesteps, n_features, n_targets, giorni_previsione):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.n_targets = n_targets
        self.giorni_previsione = giorni_previsione

    def build(self, hp):
        model = tf.keras.models.Sequential()

        # Layer LSTM iniziale
        model.add(tf.keras.layers.LSTM(hp.Int('lstm_units_1', 50, 500, step=50),
                       input_shape=(self.n_timesteps, self.n_features), kernel_initializer='glorot_uniform'))
        model.add(tf.keras.layers.RepeatVector(self.giorni_previsione))

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 4)):
            model.add(tf.keras.layers.LSTM(hp.Int(f'lstm_units_{i+2}', 50, 500, step=50), return_sequences=True,
                           kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_rate_{i+2}', 0.000001, 5, step=0.001)), kernel_initializer='glorot_uniform'))
            model.add(tf.keras.layers.Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 4)):
            model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(hp.Int(f'dense_units_{i}', 50, 500, step=50), activation='relu', kernel_initializer='glorot_uniform')))
            model.add(tf.keras.layers.Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform')))

        model.compile(optimizer="adam", 
              loss='binary_crossentropy', 
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(name='precision'), 
                       tf.keras.metrics.Recall(name='recall'), 
                       tf.keras.metrics.AUC(name='auc'),
                       tf.keras.metrics.F1Score(threshold=0.5)])
        
        return model

input_shape = (n_timesteps, n_features)
output_shape = (giorni_previsione, n_targets)
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)

def bay(hypermodel, objective, X_train, Y_train, X_val, Y_val):
    bayesian_tuner = kt.tuners.BayesianOptimization(
        hypermodel,
        objective=objective,
        max_trials=50,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()    

    bayesian_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=callbacks)
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return bayesian_tuner, best_model_bayesian, best_hyperparameters_bayesian

def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = kt.tuners.Hyperband(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    hyperband_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband

def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = kt.tuners.RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    random_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random

if not os.path.exists('dati'):
    os.makedirs('dati')

if os.path.exists("lista_ticker.parquet"):
    print("Caricamento lista_ticker esistente", flush=True)
    lista_ticker = pd.read_parquet("lista_ticker.parquet")
else:
    print("Download lista ticker", flush=True)
    lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
    lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
    lista_ticker = lista_ticker.loc[(lista_ticker["Categoria"] != "D"), :]
    lista_ticker.to_parquet("lista_ticker.parquet")

if n_simboli_addestramento == 'Tutti': 
    n_simboli_addestramento = len(lista_ticker)

objective = kt.Objective("val_f1", direction="max")

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [early_stopping, reduce_lr]

X, Y = set_di_tuning(lista_ticker)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Tuning")
print_memory_usage()
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)
print_memory_usage()
model, par, bayesian_search = bay(hypermodel, objective, X_train, Y_train, X_val, Y_val)
print_memory_usage()

trials = bayesian_search.oracle.trials

results = []

for trial in trials.values():
    trial_metrics = trial['metrics']
    val_f1 = trial_metrics['val_f1']['observations'][0]['value'] 
    results.append({
        'trial_id': trial['trial_id'],
        'val_f1': val_f1,  
        'hyperparameters': trial['hyperparameters'].values
    })

results_df = pd.DataFrame(results)
results_df.to_excel('results_df.xlsx')
    
model.save("tuning")
with open('hyperparameters.pkl', 'wb') as f:
    pickle.dump(par, f)

#addestramento(lista_ticker)
#    
print("ok")




Caricamento lista_ticker esistente
1 di 2581: Ticker OOMA
Download dati ticker


Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(68, 120, 19)
(68, 1)
2 di 2581: Ticker CE
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(92, 120, 19)
(92, 1)
3 di 2581: Ticker CHEF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(134, 120, 19)
(134, 1)
4 di 2581: Ticker ZIP
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(18, 120, 19)
(18, 1)
5 di 2581: Ticker PII
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(116, 120, 19)
(116, 1)
6 di 2581: Ticker CARR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(36, 120, 19)
(36, 1)
7 di 2581: Ticker PPBI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(120, 120, 19)
(


1 Failed download:
['MONC']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
84 di 2581: Ticker WDAY
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(100, 120, 19)
(100, 1)
85 di 2581: Ticker GPN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(100, 120, 19)
(100, 1)
86 di 2581: Ticker HTLF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(110, 120, 19)
(110, 1)
87 di 2581: Ticker STRL
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(90, 120, 19)
(90, 1)
88 di 2581: Ticker CM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(112, 120, 19)
(112, 1)
89 di 2581: Ticker MEI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(120, 120, 19)
(120, 1)
90 di 2581: Ticker PLAY
Download dati ticker



1 Failed download:
['ELN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-01-01 -> 2023-12-31)')


...


1 Failed download:
['PIRC']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
589 di 2581: Ticker K
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(94, 120, 19)
(94, 1)
590 di 2581: Ticker EXP
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(104, 120, 19)
(104, 1)
591 di 2581: Ticker HEES
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(70, 120, 19)
(70, 1)
592 di 2581: Ticker HGTY
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(6, 120, 19)
(6, 1)
593 di 2581: Ticker PCRX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(64, 120, 19)
(64, 1)
594 di 2581: Ticker 1SIE
Download dati ticker



1 Failed download:
['1SIE']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
595 di 2581: Ticker SAVE
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(70, 120, 19)
(70, 1)
596 di 2581: Ticker KBR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(146, 120, 19)
(146, 1)
597 di 2581: Ticker GCBC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(82, 120, 19)
(82, 1)
598 di 2581: Ticker SBLK
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(92, 120, 19)
(92, 1)
599 di 2581: Ticker DSGX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(104, 120, 19)
(104, 1)
600 di 2581: Ticker XRX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(98, 120, 19)
(98, 1)
Concatenamento su X, Y


: 